Azure ML & Azure Databricks notebooks by René Bremer (original taken from Ilona Stuhler and Databricks website)

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

##### In this notebook the following steps will be excuted:

1. Build model on 2000 pictures in storage account
2. Build model on dataset of 60000 pictures of full cifar-10 dataset

#0. Set parameters

In [0]:
# Folder in which the pictures are stored
mountmap="2000picscifar10"

In [0]:
dbutils.library.help()

Provides utilities for install library within notebooks.
Databricks documentation for more info.

For more info about a method, use dbutils.library.help("methodName") .
 install(path: String): boolean -> Install the library within the current notebook session installPyPI(pypiPackage: String, version: String = "", repo: String = "", extras: String = ""): boolean -> Install the PyPI library within the current notebook session list: List -> List the isolated libraries added for the current notebook session via dbutils restartPython: void -> Restart python process for the current notebook session updateCondaEnv(envYmlContent: String): boolean -> Update the current notebook's Conda environment based on the specification (content of environment

In [0]:
# LOAD libraries
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import matplotlib.pyplot as plt

import numpy as np
import os
from PIL import Image

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-224773701457054> in <module> () 
 1 # LOAD libraries 
 2 from __future__ import print_function 
 ----> 3 import keras 
 4 from keras . datasets import cifar10 
 5 from keras . preprocessing . image import ImageDataGenerator 

 /databricks/python/lib/python3.6/site-packages/keras/__init__.py in <module> () 
 19 """
 20 # pylint: disable=unused-import 
 ---> 21 from tensorflow . python import tf2 
 22 from keras import distribute 
 23 

 /databricks/python/lib/python3.6/site-packages/tensorflow/__init__.py in <module> () 
 39 import sys as _sys 
 40 
 ---> 41 from tensorflow . python . tools import module_util as _module_util 
 42 from tensorflow . python . util . lazy_loader import LazyLoader as _LazyLoader 
 43 

 /databricks/python/lib/python3.6/site-packages/tensorflow/python/__init__.py in <module> () 
 44 
 45 # Bring in subpackages. 
 ---> 46 from tensorflow . python import data 
 47 from tensorflow . python import distribute 
 48 # from tensorflow.python import keras 

 /databricks/python/lib/python3.6/site-packages/tensorflow/python/data/__init__.py in <module> () 
 23 
 24 # pylint: disable=unused-import 
 ---> 25 from tensorflow . python . data import experimental 
 26 from tensorflow . python . data . ops . dataset_ops import AUTOTUNE 
 27 from tensorflow . python . data . ops . dataset_ops import Dataset 

 /databricks/python/lib/python3.6/site-packages/tensorflow/python/data/experimental/__init__.py in <module> () 
 95 
 96 # pylint: disable=unused-import 
 ---> 97 from tensorflow . python . data . experimental import service 
 98 from tensorflow . python . data . experimental . ops . batching import dense_to_ragged_batch 
 99 from tensorflow . python . data . experimental . ops . batching import dense_to_sparse_batch 

 /databricks/python/lib/python3.6/site-packages/tensorflow/python/data/experimental/service/__init__.py in <module> () 
 351 from __future__ import print_function 
 352 
 --> 353 from tensorflow . python . data . experimental . ops . data_service_ops import distribute 
 354 from tensorflow . python . data . experimental . ops . data_service_ops import from_dataset_id 
 355 from tensorflow . python . data . experimental . ops . data_service_ops import register_dataset 

 /databricks/python/lib/python3.6/site-packages/tensorflow/python/data/experimental/ops/data_service_ops.py in <module> () 
 24 from tensorflow . python import tf2 
 25 from tensorflow . python . compat import compat 
 ---> 26 from tensorflow . python . data . experimental . ops import compression_ops 
 27 from tensorflow . python . data . experimental . ops . distribute_options import AutoShardPolicy 
 28 from tensorflow . python . data . experimental . ops . distribute_options import ExternalStatePolicy 

 /databricks/python/lib/python3.6/site-packages/tensorflow/python/data/experimental/ops/compression_ops.py in <module> () 
 18 from __future__ import print_function 
 19 
 ---> 20 from tensorflow . python . data . util import structure 
 21 from tensorflow . python . ops import gen_experimental_dataset_ops as ged_ops 
 22 

 /databricks/python/lib/python3.6/site-packages/tensorflow/python/data/util/structure.py in <module> () 
 24 import wrapt 
 25 
 ---> 26 from tensorflow . python . data . util import nest 
 27 from tensorflow . python . framework import composite_tensor 
 28 from tensorflow . python . framework import ops 

 /databricks/python/lib/python3.6/site-packages/tensorflow/python/data/util/nest.py in <module> () 
 38 import six as _six 
 39 
 ---> 40 from tensorflow . python . framework import sparse_tensor as _sparse_tensor 
 41 from tensorflow . python . util import _pywrap_utils 
 42 from tensorflow . python . util import nest 

 /databricks/python/lib/python3.6/site-packages/tensorflow/python/framework/sparse_tensor.py in <module> () 
 26 from tensorflow . python import tf2 
 27 from t

#1.  Build model on 2000 pictures in storage account

##### 1a. Load data from storage account

In [0]:
#LOAD data
n_train_data = 1800
n_test_data = 200
categoriesList=["airplane","automobile","bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
categoriesSet={"airplane":0 ,"automobile":1,"bird":2, "cat":3, "deer":4, "dog":5, "frog":6, "horse":7, "ship":8, "truck":9}

def load_via_dir(directory, n_max_datapoints, n_test):
  print("Loading data of path: " + directory)
  data_train_picture=[]
  data_train_label=[]  
  data_test_picture=[]
  data_test_label=[] 
  i=0
  for filename in os.listdir(directory):
      if filename.endswith(".png"): 
        #print(os.path.join(directory, filename))
        picture = np.asarray(Image.open(os.path.join(directory, filename)))
        name, numpng = filename.split("_",2)
        num, png = numpng.split(".",1) 
        n_max_train = (n_max_datapoints - n_test)/len(categoriesSet)
        if int(num) < n_max_train + 1:        
          data_train_picture.append(picture)
          data_train_label.append(categoriesSet[name])
        else:
          data_test_picture.append(picture)
          data_test_label.append(categoriesSet[name])        
        
        if i % 100 == 0 and i != 0:
           print(str(i)+ " of " + str(n_train_data + n_test_data) + " pictures loaded.")
        i=i+1
        
        if(n_max_datapoints==len(data_train_picture) + len(data_test_picture)):
          print(str(len(data_train_picture) + len(data_test_picture))+" pictures loaded.")
          return data_train_picture, data_train_label, data_test_picture, data_test_label
        continue
      else:
          continue
          
data_train_picture, data_train_label, data_test_picture, data_test_label = load_via_dir("/dbfs/mnt/" + mountmap + "/", n_train_data+n_test_data, n_test_data)

In [0]:
# The data, shuffled and split between train and test sets:
x_train = np.asarray(data_train_picture)
y_train = np.asarray(data_train_label)
x_test = np.asarray(data_test_picture)
y_test = np.asarray(data_test_label)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

##### 1b. Initial Model Training ParametersParameter Setup

1. **Batch Size** - Defines the number of samples that will be propagated through the network.<br>
    We will use a batch size of **32** in this Lab
    
2. **Number of Classes**<br>
    CIFAR10 Dataset has **10** target classes
    
3. **Epochs** - Number times that the learning algorithm will run through the entire training dataset.<br>
    We will use **30 Epochs** in this lab to reduce the overall training time

In [0]:
batch_size = 32
num_classes = 10
epochs = 30

##### 1c. Preprocessing Steps
1. Convert class vectors to binary class matrices
2. Cast PIXEL Values to FLOAT
3. Normalize Pixel RGB Values to (0:1)

In [0]:
#1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
#2
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
#3
x_train /= 255
x_test /= 255

##### 1d. Define Convolution Network

In [0]:
def initializeModel():
  model = Sequential()

  model.add(Conv2D(32, (3, 3), padding='same',
                   input_shape=x_train.shape[1:]))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))
  return model


In [0]:
model2000 = initializeModel()

In [0]:
## Print Model Summary
model2000.summary()

##### 1e. Define Optimizers and Compile the model

**Optimization algorithms** are used to minimize (or maximize) an Objective function (also called Error function) E(x) which is mathematical function dependent on the Model’s internal learnable parameters that are used in computing the target values(Y) from the set of predictors(X) used by the machine learning model. In neural networks the Weights(W) and the Bias(b) values are the learnable parameters which are used in computing the output values and are learned and updated in the direction of best solution i.e minimizing the Loss by the network’s training process.

Below are som of the most used optimizers available in Keras:
1. Stochastic Gradient Descent (SGD)
2. Root Mean Square Propagation (RMSProp)
3. Adaptive Gradient (ADAGrad)
4. Adaptive Moment Estimation (ADAM)
  
**Hyper Parameters**
  Two hyper parameters used by most optimizers are:
  1. Learning Rate - Learning rate (lr) controls the magnitude of adjustment of the weights of the neural network with respect the loss gradient.
  2. Decay - Weight decay is a regularization term that causes weights to exponentially decay to zero and hence penalizes big weights.

In [0]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model2000.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

##### 1f.  Model Training : without Data Augmentation (Takes 1 Min)

In [0]:
from keras.callbacks import History 
history_noAug = History()
print('Not using data augmentation.')
model2000.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True,
         callbacks=[history_noAug],
         verbose=1)

##### 1g. Evaluate model

In [0]:
score = model2000.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', score[1])
print('Test loss:', score[0])

In [0]:
def plotmetrics(history):
  width = 10
  height = 5
  ## Clear plot if repeated call
  plt.clf()
  plt.figure(figsize=(width, height))
  # Plot training & validation accuracy values
  plt.title('Model Metrics : Non Augmented Data')
  plt.subplot(1, 2, 1)
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  # Plot training & validation loss values
  plt.subplot(1, 2, 2)
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  pltoutput = plt.show()
  return pltoutput

In [0]:
# Visualize model
pltoutput = plotmetrics(history_noAug)
display(pltoutput)

##### 1h. Show predictions in plot

In [0]:
categoriesList=["airplane","automobile","bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

import matplotlib.pyplot as plt
import random
def plotImages(x_test, images_arr, labels_arr, n_images=8):
    fig, axes = plt.subplots(n_images, n_images, figsize=(9,9))
    axes = axes.flatten()
    
    for i in range(100):
        rand = random.randint(0, x_test.shape[0] -1)
        img = images_arr[rand]
        ax = axes[i]
    
        ax.imshow( img, cmap="Greys_r")
        ax.set_xticks(())
        ax.set_yticks(())
        
        predictions = model2000.predict_classes([[x_test[rand]]])
        label=categoriesList[predictions[0]]   
        
        if labels_arr[rand][predictions[0]] == 0:
            ax.set_title(label, fontsize=18 - n_images, color="red") 
        else:
            ax.set_title(label, fontsize=18 - n_images) 
        
    plot = plt.tight_layout()
    return plot
  
display(plotImages(x_test, data_test_picture, y_test, n_images=10))

##### 1i. Save Trained Model (without Data Augmentation) to DBFS as HDF5 file

In [0]:
## Create Output Model Directory
dbutils.fs.mkdirs('/CIFAR10/models/')

In [0]:
from keras.models import load_model
modelpath = '/dbfs/CIFAR10/models/cifar_2000pictures.h5'
model2000.save(modelpath)

#model = load_model(modelpath)

#2. Build model on dataset of 60000 pictures of full cifar-10 dataset

##### 2a. Load data directly from cifar-10 website using cifar10 library in Keras

In [0]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

##### 2b. Initial Model Training ParametersParameter Setup

1. **Batch Size** - Defines the number of samples that will be propagated through the network.<br>
    We will use a batch size of **32** in this Lab
    
2. **Number of Classes**<br>
    CIFAR10 Dataset has **10** target classes
    
3. **Epochs** - Number times that the learning algorithm will run through the entire training dataset.<br>
    We will use **30 Epochs** in this lab to reduce the overall training time

In [0]:
batch_size = 32
num_classes = 10
epochs = 30

##### 2c. Preprocessing Steps
1. Convert class vectors to binary class matrices
2. Cast PIXEL Values to FLOAT
3. Normalize Pixel RGB Values to (0:1)

In [0]:
#1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
#2
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
#3
x_train /= 255
x_test /= 255

##### 2d. Define Convolution Network

In [0]:
def initializeModel():
  model = Sequential()

  model.add(Conv2D(32, (3, 3), padding='same',
                   input_shape=x_train.shape[1:]))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))
  return model

In [0]:
modelfull = initializeModel()

In [0]:
modelfull.summary()

##### 2e. Define Optimizers and Compile the model

**Optimization algorithms** are used to minimize (or maximize) an Objective function (also called Error function) E(x) which is mathematical function dependent on the Model’s internal learnable parameters that are used in computing the target values(Y) from the set of predictors(X) used by the machine learning model. In neural networks the Weights(W) and the Bias(b) values are the learnable parameters which are used in computing the output values and are learned and updated in the direction of best solution i.e minimizing the Loss by the network’s training process.

Below are som of the most used optimizers available in Keras:
1. Stochastic Gradient Descent (SGD)
2. Root Mean Square Propagation (RMSProp)
3. Adaptive Gradient (ADAGrad)
4. Adaptive Moment Estimation (ADAM)
  
**Hyper Parameters**
  Two hyper parameters used by most optimizers are:
  1. Learning Rate - Learning rate (lr) controls the magnitude of adjustment of the weights of the neural network with respect the loss gradient.
  2. Decay - Weight decay is a regularization term that causes weights to exponentially decay to zero and hence penalizes big weights.

In [0]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
modelfull.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

##### 2f.  Model Training : without Data Augmentation (Takes 5-8 Mins)

In [0]:
from keras.callbacks import History 
history_noAug = History()
print('Not using data augmentation.')
modelfull.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True,
         callbacks=[history_noAug],
         verbose=1)

##### 2g. Evaluate model

In [0]:
score = modelfull.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', score[1])
print('Test loss:', score[0])

In [0]:
def plotmetrics(history):
  width = 10
  height = 5
  ## Clear plot if repeated call
  plt.clf()
  plt.figure(figsize=(width, height))
  # Plot training & validation accuracy values
  plt.title('Model Metrics : Non Augmented Data')
  plt.subplot(1, 2, 1)
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  # Plot training & validation loss values
  plt.subplot(1, 2, 2)
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  pltoutput = plt.show()
  return pltoutput

In [0]:
pltoutput = plotmetrics(history_noAug)
display(pltoutput)

##### 2h. Show predictions in plot

In [0]:
categoriesList=["airplane","automobile","bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

import matplotlib.pyplot as plt
import random
def plotImages(x_test, images_arr, labels_arr, n_images=8):
    fig, axes = plt.subplots(n_images, n_images, figsize=(9,9))
    axes = axes.flatten()
    
    for i in range(100):
        rand = random.randint(0, x_test.shape[0] -1)
        img = images_arr[rand]
        ax = axes[i]
    
        ax.imshow( img, cmap="Greys_r")
        ax.set_xticks(())
        ax.set_yticks(())
        
        predictions = modelfull.predict_classes([[x_test[rand]]])
        
        label=categoriesList[predictions[0]]   
        
        print(predictions[0])
        print(labels_arr[rand])
        
        if labels_arr[rand][predictions[0]] == 0:
            ax.set_title(label, fontsize=18 - n_images, color="red") 
        else:
            ax.set_title(label, fontsize=18 - n_images) 
        
    plot = plt.tight_layout()
    return plot
  
display(plotImages(x_test, x_test, y_test, n_images=10))

##### 1i. Save Trained Model (without Data Augmentation) to DBFS as HDF5 file

In [0]:
from keras.models import load_model
modelpath = '/dbfs/CIFAR10/models/cifar_allpictures.h5'
#modelfull.save(modelpath)
modelfull = load_model(modelpath)

In [0]:
predictions = modelfull.predict_classes([[x_test[80]]])
print(y_test[0][4])